In [59]:
hsn_code = ['260300']
years = ['2021', '2022', '2023', '2024']

In [73]:
df1 = get_trade_qty(hsn_code, years)
df2 = get_trade_value_USD(hsn_code, years)

In [76]:
df = pd.merge(df1, df2, on=['HS_Code','Year'])

In [ ]:
# Save it in postgres database

from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
from dotenv import load_dotenv
import math

# Load environment variables
load_dotenv(override=True)

# Define the PostgreSQL connection string
connection_string = os.getenv('POSTGRES_DB_URL')

# Create the SQLAlchemy engine for PostgreSQL
engine = create_engine(connection_string)

# Define the base for the declarative mapping
Base = declarative_base()

# Define the table schema for PostgreSQL
class AuctionData(Base):
    __tablename__ = 'trade_data'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Auto-incrementing ID
    hs_code = Column(String(255))
    start_date = Column(Date)
    end_date = Column(Date)
    import_qty = Column(Float)
    export_qty = Column(Float)
    import_value_usd = Column(Float)
    export_value_usd = Column(Float)

# Create the table in the PostgreSQL database
Base.metadata.create_all(engine)

# Prepare your DataFrame (scraped data already processed in df)

# Define a session to interact with the PostgreSQL database
Session = sessionmaker(bind=engine)
session = Session()

# Convert the DataFrame to a list of dictionaries, compatible with SQLAlchemy ORM
auction_data_dicts = df.to_dict(orient='records')

# Insert data into the auction_data table
for record in auction_data_dicts:
    # Replace nan with None for PostgreSQL compatibility
    record = {key: (None if (isinstance(value, float) and math.isnan(value)) else value) 
              for key, value in record.items()}
    
    auction_entry = AuctionData(**record)  # Unpack the dictionary into the ORM object
    session.add(auction_entry)

# Commit the transaction
session.commit()

print("Data inserted successfully into PostgreSQL!")

# Close the session
session.close()


In [2]:
url_import = 'https://tradestat.commerce.gov.in/eidb/icomq.asp'
url_export = 'https://tradestat.commerce.gov.in/eidb/ecomq.asp'

In [54]:
# importing Library 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from datetime import datetime
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
from dotenv import load_dotenv
import math

In [56]:
# Open the testing browser
chrome_options = Options()
chrome_options.use_chromium = True

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [57]:
def get_import_qty(hsn_code, years):
    driver.get(url_import)
    data = []
    for code in hsn_code:
        for year in years:
            select_element_year = driver.find_element(By.ID, "select2")
            dropdown_year = Select(select_element_year)
    
            # Selecting the year for which we need to fetch data
            dropdown_year.select_by_value(year)
    
            input = driver.find_element(By.NAME, 'hscode')
            input.clear()
            input.send_keys(code)
            
            select_element = driver.find_element(By.ID, "select1")
            dropdown = Select(select_element)
            
            # Select hsn_code digit
            dropdown.select_by_value(str(len(code)))
            
            # Quantity is available for 6 & 8 digit HSN code
            if len(code) > 4:

                # Pressing Radio button
                qty = driver.find_element(By.ID, 'radioqty')
                qty.click()

                # Submit
                submit_btn = driver.find_element(By.ID, 'button1')
                submit_btn.click()
    
                # Scrap the table data
                table_data = driver.find_elements(By.TAG_NAME, 'td')
                table_header = driver.find_elements(By.TAG_NAME, 'th')

                
                hs_code = code
                y1 = int(year) - 1
                y2 = year
                year_text = str(y1)+'-'+str(y2)
                qty_text = 0

                if table_header:
                    arr_th = [th.text for th in table_header]
                    year_text = arr_th[4]

                if table_data:
                    arr_td = [td.text for td in table_data]
                    qty_text = arr_td[4].replace(',', '') if arr_td[4] != ' ' else 0

                if qty_text == '' or qty_text == ' ':
                    qty_text = 0

                
                # Add the row to the data list
                row = [year_text, hs_code, float(qty_text)]
                data.append(row)
                
                # Back Button
                back_btn = driver.find_element(By.ID, 'IMG1')
                back_btn.click()
    
    df = pd.DataFrame(data, columns=['Year', 'HS_Code', 'Import_Qty_thousand_units'])
       
    return df

In [58]:
def get_export_qty(hsn_code, years):
    driver.get(url_export)
    data = []
    for code in hsn_code:
        for year in years:
            select_element_year = driver.find_element(By.ID, "select2")
            dropdown_year = Select(select_element_year)
    
            # Selecting the year for which we need to fetch data
            dropdown_year.select_by_value(year)
    
            input = driver.find_element(By.NAME, 'hscode')
            input.clear()
            input.send_keys(code)
            
            select_element = driver.find_element(By.ID, "select1")
            dropdown = Select(select_element)
            
            # Select hsn_code digit
            dropdown.select_by_value(str(len(code)))
            
            # Quantity is available for 6 & 8 digit HSN code
            if len(code) > 4:

                # Pressing Radio button
                qty = driver.find_element(By.ID, 'radioqty')
                qty.click()

                # Submit
                submit_btn = driver.find_element(By.ID, 'button1')
                submit_btn.click()
    
                # Scrap the table data
                table_data = driver.find_elements(By.TAG_NAME, 'td')
                table_header = driver.find_elements(By.TAG_NAME, 'th')

                
                hs_code = code
                y1 = int(year) - 1
                y2 = year
                year_text = str(y1)+'-'+str(y2)
                qty_text = 0

                if table_header:
                    arr_th = [th.text for th in table_header]
                    year_text = arr_th[4]

                if table_data:
                    arr_td = [td.text for td in table_data]
                    qty_text = arr_td[4].replace(',', '') if arr_td[4] != ' ' else 0

                if qty_text == '' or qty_text == ' ':
                    qty_text = 0

                
                # Add the row to the data list
                row = [year_text, hs_code, float(qty_text)]
                data.append(row)
                
                # Back Button
                back_btn = driver.find_element(By.ID, 'IMG1')
                back_btn.click()
    
    df = pd.DataFrame(data, columns=['Year', 'HS_Code', 'Export_Qty_thousand_units'])
       
    return df

In [26]:
# Merge import and export data into one dataframe
def get_trade_qty(hsn_code, years):
    df1 = get_import_qty(hsn_code, years)
    df2 = get_export_qty(hsn_code, years)
    merged_df = pd.merge(df1, df2, on=['Year','HS_Code'])
    return merged_df



In [62]:
# get trade import value US $
def get_import_value_USD(hsn_code, years):
    driver.get(url_import)
    data = []
    for code in hsn_code:
        for year in years:
            select_element_year = driver.find_element(By.ID, "select2")
            dropdown_year = Select(select_element_year)
    
            # Selecting the year for which we need to fetch data
            dropdown_year.select_by_value(year)
    
            input = driver.find_element(By.NAME, 'hscode')
            input.clear()
            input.send_keys(code)
            
            select_element = driver.find_element(By.ID, "select1")
            dropdown = Select(select_element)
            
            # Select hsn_code digit
            dropdown.select_by_value(str(len(code)))
            
    
            # Pressing USD Radio button
            qty = driver.find_element(By.ID, 'radiousd')
            qty.click()

            # Pressing Submit Button
            submit_btn = driver.find_element(By.ID, 'button1')
            submit_btn.click()
            
            header = driver.find_elements(By.TAG_NAME, 'th')
            table_data = driver.find_elements(By.TAG_NAME, 'td')

            y1 = int(year)-1
            y2 = year
            year_text = str(y1)+'-'+str(y2)
            value_text = '0'
            
            if header:
                arr_hd = [i for i in header]
                year_text = arr_hd[3].text
            
            if table_data:
                arr_td = [i for i in table_data]
                value_text = arr_td[3].text.replace(',','').strip()
            
            if value_text == '' or value_text == ' ':
                value_text = '0'

            row = [year_text, code, float(value_text)]
            data.append(row)
            
            # Back Button
            back_btn = driver.find_element(By.ID, 'IMG1')
            back_btn.click()  
    
    # Convert the array into a Dataframe
    return pd.DataFrame(data, columns=['Year', 'HS_Code', 'Import_USD_millions'])

In [61]:
# get trade import value US $
def get_export_value_USD(hsn_code, years):
    driver.get(url_export)
    data = []
    for code in hsn_code:
        for year in years:
            select_element_year = driver.find_element(By.ID, "select2")
            dropdown_year = Select(select_element_year)
    
            # Selecting the year for which we need to fetch data
            dropdown_year.select_by_value(year)
    
            input = driver.find_element(By.NAME, 'hscode')
            input.clear()
            input.send_keys(code)
            
            select_element = driver.find_element(By.ID, "select1")
            dropdown = Select(select_element)
            
            # Select hsn_code digit
            dropdown.select_by_value(str(len(code)))
            
    
            # Pressing USD Radio button
            qty = driver.find_element(By.ID, 'radiousd')
            qty.click()

            # Pressing Submit Button
            submit_btn = driver.find_element(By.ID, 'button1')
            submit_btn.click()
            
            header = driver.find_elements(By.TAG_NAME, 'th')
            table_data = driver.find_elements(By.TAG_NAME, 'td')

            y1 = int(year)-1
            y2 = year
            year_text = str(y1)+'-'+str(y2)
            value_text = '0'
            
            if header:
                arr_hd = [i for i in header]
                year_text = arr_hd[3].text
            
            if table_data:
                arr_td = [i for i in table_data]
                value_text = arr_td[3].text.replace(',','').strip()
            
            if value_text == '' or value_text == ' ':
                value_text = '0'

            row = [year_text, code, float(value_text)]
            data.append(row)
            
            # Back Button
            back_btn = driver.find_element(By.ID, 'IMG1')
            back_btn.click()  
    
    # Convert the array into a Dataframe
    return pd.DataFrame(data, columns=['Year', 'HS_Code', 'Export_USD_millions)'])

In [52]:
# merge trade_value USD data
def get_trade_value_USD(hsn_code, years):
    df1 = get_import_value_USD(hsn_code, years)
    df2 = get_export_value_USD(hsn_code, years)
    merged_df = pd.merge(df1, df2, on=['Year','HS_Code'])
    return merged_df